# AYCE Calculation (Data Input & Output):

In this simple example, we define a json of a "Banana Jam" product that we will send to the EOS server for calculation.

Please make sure to install `ipycytoscape` in your python environment to vizualize the resulting LCA graph.
For example, if you use pip, execute:
```
    pip install ipycytoscape
```

You can then click on nodes in the resulting graph to see the corresponding properties of that node.

In [1]:
import uuid
import requests
from ipywidgets import HTML, HBox
from ipycytoscape import CytoscapeWidget

In [2]:
# Define a product as input using ingredients_declaration:
banana_jam_json = {
  "date": "2023-09-06",
  "names": [
    {
      "language": "en",
      "value": "Banana Jam"
    }
  ],
  "amount": 100,
  "unit": "gram",
  "ingredients_declaration":  [
    {
      "language": "en",
      "value": "Bananas, Lemon Juice Concentrate."
    }
  ],
  "nutrient_values": {
    "energy_kcal": 100,
    "fat_gram": 0,
    "saturated_fat_gram": 0,
    "carbohydrates_gram": 25,
    "sucrose_gram": 18,
    "protein_gram": 0,
    "sodium_chloride_gram": 0
  },
  "location": "Switzerland",
}

# Send to the EOS calculation engine
This Python code sends product data for "Banana Jam" to the EOS calculation engine for environmental analysis. 
Ensure the required libraries and authentication tokens are set up before running the code.

In [3]:
EOS_BASE_PATH = "https://eos.eaternity.ch"
EOS_AUTH_TOKEN = "443f45f4-1bbd-43ee-bc6d-9ce06584a217"

node_uid = uuid.uuid4()
response = requests.put(
    f"{EOS_BASE_PATH}/v2/calculation/node-uid/{node_uid}?return_final_graph=true",
    json={
        "node": {
            "node_type": 'food_product',
            "props": {
                "raw_input": [{
                    "data": banana_jam_json
                }]
            }
        }
    },
    auth=requests.auth.HTTPBasicAuth(EOS_AUTH_TOKEN, '')
)
assert response.status_code == 200, f"Error posting calculation: {response.text}, {response.status_code}"

# Print Result
First print the overall calculated kg CO2eq value of the root node.

Then also print potential `data_errors` that happened during the calculation.

In [4]:
resp = response.json()
root_node = resp["node"]
print(f"kg co2eq: {root_node.get('co2-value')}")

print("\ndata_errors: ")
for line in resp.get("calculation").get("data_errors"):
    print("\n"+line)

kg co2eq: 0.11971698416248285

data_errors: 

Could not match product name [{'value': 'Banana Jam', 'language': 'en'}] in node NodeView(Activity {'type': 'food_product', 'raw_input.names': 'Banana Jam', 'raw_input.location': 'Switzerland', 'raw_input.amount': 100}) (is compound product: True)


# Restructuring the LCA Graph for Display
In this section, we are reformatting the LCA (Life Cycle Assessment) graph retrieved from the JSON response for later visualization in Cytoscape. 

The code organizes nodes and edges, assigns names and background colors to nodes, and creates connections between nodes for improved graph representation.

In [5]:
nodes = resp.get("calculation").get("final_graph")
nodes_by_uid = { node["uid"]: node for node in nodes }
edges = []
for node in nodes:
    name = node["props"].get("raw_input", [{}])[0].get("data", {}).get("names", [{}])[0].get("value")
    if not name:
        name = node["props"].get("product_name", [{}])[0].get("term_name", "")
    node['data'] = {
        'id': node['uid'],
        'name': name,
        'bgcolor': 'green' if node["node_type"] in ("food_product") else "red"
    }
    for sub_node_uid in node['sub_node_uids']:
        edges.append({
            'data': {
                'source': node.get('uid'),
                'target': sub_node_uid
            },
        })

# Configuring and Visualizing the LCA Graph
In this section, we configure the visualization of the LCA (Life Cycle Assessment) graph using Cytoscape. 

The code sets up the graph layout, style, and interaction features. 
Users can click on nodes to display their properties in a separate panel.

In [6]:
cyto = CytoscapeWidget(layout={
    'border': '1px solid black'
})
cyto.graph.add_graph_from_json({
    "nodes": nodes,
    "edges": edges
})
def edgeWeight(edge):
    return 1
cyto.set_layout(
    name='dagre',
    spacingFactor=1.0,
    ranker="longest-path",
    boundingBox={
        "x1": 0,
        "y1": 0,
        "w": 2000,
        "h": 2000,
    },
    rankDir="LR", 
    nodeDimensionsIncludeLabels=True)
cyto.set_style([
    {
        'selector': 'node',
        'css': {
            'background-color': 'data(bgcolor)',
            'content': 'data(name)',
            'text-valign': 'center',
        }
    }
])

# Now we make the graph nodes clickable and show the corresponding node properties in another panel:
prop_panel = HTML("<i>Please click on a node to see it's properties...</i>")
def log_clicks(node):
    prop_panel.value = "" # clear first
    selected_node = nodes_by_uid[node["data"]["id"]]
    for prop_name, prop_val in sorted(selected_node["props"].items()):
        prop_panel.value += f'<p><b>{prop_name}:</b></p>'
        prop_panel.value += f'<div style="max-height:120px;overflow-y:auto;">{prop_val}</div>'
cyto.on('node', 'click', log_clicks)

# Visualizing the Graph

In this section, we utilize an HBox layout to present the graph on the left side and the properties of clicked nodes on the right side.

- <span style="font-weight: bold;color:green">Green Nodes</span>: Represent 'activities' within the graph.
- <span style="font-weight: bold;color:red">Red Nodes</span>: Represent 'flows' within the graph.
- <span style="font-weight: bold;color:gray">Gray Nodes</span>: Indicate data not included in the response due to proprietary information from ecoinvent.

In [7]:
sidebyside = HBox([cyto, prop_panel])
display(sidebyside)
cyto.layout.width='50%'
cyto.layout.height='800px'
cyto.layout.max_height='100%'
cyto.layout.max_width='50%'
prop_panel.layout.width='50%'